In [7]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
import sklearn
from numpy import inf

In [10]:
%run structured.ipynb

In [11]:
#from fastai.tabular import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import seaborn as sns
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
#from workalendar.europe import Russia
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import warnings

In [12]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [13]:
item_cat = pd.read_csv('/users/youcefdjeddar/downloads/sales/item_categories.csv')
items = pd.read_csv('/users/youcefdjeddar/downloads/sales/items.csv')
sales = pd.read_csv('/users/youcefdjeddar/downloads/sales/sales_train.csv')
shops = pd.read_csv('/users/youcefdjeddar/downloads/sales/shops.csv')
test = pd.read_csv('/users/youcefdjeddar/downloads/sales/test.csv')
submi = pd.read_csv('/users/youcefdjeddar/downloads/sales/sample_submission.csv')

In [14]:
len(item_cat) + len(items) + len(sales) + len(shops) + len(test)

3172363

In [14]:
sales.date = pd.to_datetime(sales.date)

In [21]:
sales['day'] = sales.date.dt.day
sales['month'] = sales.date.dt.month
sales['year'] = sales.date.dt.year

In [22]:
sales['revenue'] = sales['item_price'] * sales['item_cnt_day']

In [23]:
sales.head(1)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,day,month,year
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013


In [24]:
sales_merged = pd.merge(sales, items, how = 'left', on = 'item_id')

In [25]:
sales_merged = pd.merge(sales_merged, shops, how = 'left', on = 'shop_id')

In [26]:
sales_merged.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,day,month,year,item_name,item_category_id,shop_name
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир"""
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум"""


In [89]:
#cal = Russia()
#holidays = cal.holidays(2013) ; holidays

In [27]:
holidays = ['2013-01-01', '2014-01-01', '2015-01-01', '2013-01-02', '2014-01-02', 
            '2015-01-02', '2013-01-07', '2014-01-07', '2015-01-07', '2013-02-23', '2014-02-23', '2015-02-23',
            '2013-03-08', '2014-03-08', '2015-03-08', '2013-05-01', '2014-05-01', '2015-05-01', '2013-05-09', 
           '2014-05-09', '2015-05-09', '2013-06-12', '2014-06-12', '2015-06-12', '2013-11-04', '2014-11-04', '2015-11-04']

In [28]:
sales_merged['holiday'] = sales_merged['date'].dt.date.astype('datetime64').isin(holidays)

In [29]:
sales_merged['weekday'] = pd.to_datetime(sales_merged['date']).dt.dayofweek  

In [30]:
sales_merged['weekend'] = 0
sales_merged.loc[sales_merged['weekday'].isin([5, 6]), 'weekend'] = 1  # 5 and 6 correspond to Sat and Sun

In [31]:
sales_merged.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",False,4,0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",False,4,0


In [32]:
#Item Count Per Month Per Shop is X
x = sales_merged.groupby(by=[sales_merged.month, sales_merged.year, sales_merged.shop_id])['item_cnt_day'].sum()

In [33]:
x = pd.DataFrame(x)

In [34]:
x.head(1)

,,,item_cnt_day
month,year,shop_id,
1,2013,0,3626.0


In [36]:
df = pd.merge(sales_merged, x, how = 'left', on = ['shop_id', 'month', 'year']) ; df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_day_y
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",False,4,0,2063.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",False,4,0,9051.0


In [37]:
df = df.rename(columns = {'item_cnt_day_y': 'item_cnt_month', 'item_cnt_day_x':'item_cnt_day'})

In [38]:
df['holiday'] = LabelEncoder().fit_transform(df['holiday'])

In [39]:
df.head(1)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0


In [40]:
product_category_month = df.groupby(by=[df.month, df.year, df.shop_id, df.item_category_id])['item_cnt_day'].sum()

In [41]:
product_category_month.head(2)

month  year  shop_id  item_category_id
1      2013  0        2                   28.0
                      3                   18.0
Name: item_cnt_day, dtype: float64

In [42]:
product_category_month = pd.DataFrame(product_category_month)

In [43]:
df = pd.merge(df, product_category_month, how = 'left', on = ['shop_id', 'month', 'year', 'item_category_id'])

In [44]:
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,item_cnt_day_y
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0


In [45]:
df = df.rename(columns = {'item_cnt_day_y': 'cat_cnt_month', 'item_cnt_day_x':'item_cnt_day'})

In [46]:
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0


#### Sanity Check

In [47]:
x = df[(df['shop_id'] == 59) & (df['month'] == 2) & (df['year'] == 2013)] ; x.head(1)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0


In [48]:
sum(x.item_cnt_day)

2063.0

#### End Sanity Check

In [49]:
x = df.groupby(by=[df.month, df.year, df.shop_id, df.item_id])['item_cnt_day'].sum() ; x = pd.DataFrame(x) ; x.head(4)

item_cnt_day
month year shop_id item_id              
1     2013 0       32                4.0
                   33                2.0
                   35                2.0
                   43                1.0

In [50]:
df = pd.merge(df, x, how = 'left', on = ['month', 'year', 'shop_id', 'item_id']) 

In [51]:
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_day_y
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0,1.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0,1.0


In [52]:
df = df.rename(columns = {'item_cnt_day_x':'item_cnt_day', 'item_cnt_day_y':'item_cnt_month_shop'})

In [53]:
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0,1.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0,1.0


In [54]:
revenue_month_shop = df.groupby(by=[df.month, df.year, df.shop_id])['revenue'].sum() ; revenue_month_shop = pd.DataFrame(revenue_month_shop) ; revenue_month_shop.head(2)

revenue
month year shop_id           
1     2013 0        1852216.0
           1         962836.0

In [55]:
df = pd.merge(df, revenue_month_shop, how = 'left', on = ['month', 'year', 'shop_id']) 

In [56]:
df = df.rename(columns = {'revenue_x':'revenue_day', 'revenue_y':'revenue_month'})

In [57]:
df.head(3)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,year,item_name,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0,1.0,1485324.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0,1.0,6122578.2
2,2013-05-01,0,25,2552,899.0,-1.0,-899.0,1,5,2013,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",1,2,0,7641.0,72.0,-1.0,5162153.6


In [58]:
num_item_month_shop = df.groupby(by=[df.month, df.year, df.shop_id])['item_cnt_day'].sum() ; num_item_month_shop = pd.DataFrame(num_item_month_shop) ; num_item_month_shop.head(4)

item_cnt_day
month year shop_id              
1     2013 0              3626.0
           1              1955.0
           2               748.0
           3               670.0

In [59]:
df = pd.merge(df, num_item_month_shop, how = 'left', on = ['month', 'year', 'shop_id']) ; df.head(3)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,revenue_day,day,month,year,...,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,item_cnt_day_y
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,...,37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0,1.0,1485324.0,2063.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,...,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0,1.0,6122578.2,9051.0
2,2013-05-01,0,25,2552,899.0,-1.0,-899.0,1,5,2013,...,58,"Москва ТРК ""Атриум""",1,2,0,7641.0,72.0,-1.0,5162153.6,7641.0


In [60]:
df = df.rename(columns = {'item_cnt_day_x':'item_cnt_day', 'item_cnt_day_y':'global_cnt_month_shop'})

In [61]:
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,year,...,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,global_cnt_month_shop
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,...,37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0,1.0,1485324.0,2063.0
1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,2013,...,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0,1.0,6122578.2,9051.0


In [62]:
x = df[(df['shop_id'] == 59) & (df['month'] == 2) & (df['year'] == 2013)] ; x.head(1)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,year,...,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,global_cnt_month_shop
0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,2013,...,37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0,1.0,1485324.0,2063.0


In [63]:
sum(x.revenue_day)

1485323.9999999988

In [64]:
df.to_csv('/users/youcefdjeddar/downloads/sales/sales_refined.csv')

In [119]:
df = pd.read_csv('/users/youcefdjeddar/downloads/sales/sales_refined.csv', index_col=False, low_memory=False)

In [120]:
df.head(3)

,Unnamed: 0,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,...,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,global_cnt_month_shop
0,0,2013-02-01,0,59,22154,999.0,1.0,999.0,1,2,...,37,"Ярославль ТЦ ""Альтаир""",0,4,0,2063.0,111.0,1.0,1485324.0,2063.0
1,1,2013-03-01,0,25,2552,899.0,1.0,899.0,1,3,...,58,"Москва ТРК ""Атриум""",0,4,0,9051.0,66.0,1.0,6122578.2,9051.0
2,2,2013-05-01,0,25,2552,899.0,-1.0,-899.0,1,5,...,58,"Москва ТРК ""Атриум""",1,2,0,7641.0,72.0,-1.0,5162153.6,7641.0


### Advanced Feature Engineering

In [121]:
min_max_scaler = preprocessing.MinMaxScaler()
rfr = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 10, max_depth = 10, n_jobs = -1, oob_score = True)

In [122]:
df_sorted = df.sort_values(by=['date'])

In [123]:
df_sorted.head(2)

,Unnamed: 0,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,...,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,global_cnt_month_shop
109593,109593,2013-01-01,0,46,18616,349.0,1.0,349.0,1,1,...,40,"Сергиев Посад ТЦ ""7Я""",1,1,0,2749.0,656.0,1.0,1837344.00,2749.0
85162,85162,2013-01-01,0,54,11854,199.0,1.0,199.0,1,1,...,63,"Химки ТЦ ""Мега""",1,1,0,7814.0,87.0,1.0,4843727.58,7814.0


In [124]:
train_df = df_sorted.loc[df_sorted['shop_id'].isin(test['shop_id'])]

In [125]:
train_df = train_df.loc[train_df['item_id'].isin(test['item_id'])] ; train_df.head(2)

,Unnamed: 0,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,...,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,global_cnt_month_shop
18128,18128,2013-01-01,0,28,4906,1799.0,1.0,1799.0,1,1,...,19,"Москва ТЦ ""МЕГА Теплый Стан"" II",1,1,0,6058.0,377.0,8.0,3840464.53,6058.0
47143,47143,2013-01-01,0,15,3686,899.0,1.0,899.0,1,1,...,19,"Калуга ТРЦ ""XXI век""",1,1,0,2353.0,157.0,1.0,1769615.00,2353.0


In [61]:
#train_df = train_df.iloc[:, :-1]

In [126]:
train_df = train_df.rename(columns = {'Unnamed: 0':'ID'})

In [127]:
train_df.head(3)

,ID,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,...,item_category_id,shop_name,holiday,weekday,weekend,item_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,global_cnt_month_shop
18128,18128,2013-01-01,0,28,4906,1799.0,1.0,1799.0,1,1,...,19,"Москва ТЦ ""МЕГА Теплый Стан"" II",1,1,0,6058.0,377.0,8.0,3840464.53,6058.0
47143,47143,2013-01-01,0,15,3686,899.0,1.0,899.0,1,1,...,19,"Калуга ТРЦ ""XXI век""",1,1,0,2353.0,157.0,1.0,1769615.00,2353.0
107443,107443,2013-01-01,0,37,22087,79.0,2.0,158.0,1,1,...,83,"Новосибирск ТЦ ""Мега""",1,1,0,1959.0,26.0,11.0,1312509.00,1959.0


In [128]:
train_df.columns

Index(['ID', 'date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'revenue_day', 'day', 'month', 'year', 'item_name',
       'item_category_id', 'shop_name', 'holiday', 'weekday', 'weekend',
       'item_cnt_month', 'cat_cnt_month', 'item_cnt_month_shop',
       'revenue_month', 'global_cnt_month_shop'],
      dtype='object')

### Fastai Approach

In [469]:
cat_vars = ['date_block_num', 'shop_id', 'item_id', 'day', 'month', 'year', 
            'item_category_id', 'holiday', 'weekday', 'weekend']
contin_vars = ['item_price', 'revenue_day', 'item_cnt_day', 'item_cnt_month', 'cat_cnt_month', 'revenue_month', 'global_cnt_month_shop']

In [470]:
dep = 'item_cnt_month_shop' 
joined = train_df[cat_vars+contin_vars+[dep]].copy()

In [471]:
joined.head(3)

,date_block_num,shop_id,item_id,day,month,year,item_category_id,holiday,weekday,weekend,item_price,revenue_day,item_cnt_day,item_cnt_month,cat_cnt_month,revenue_month,global_cnt_month_shop,item_cnt_month_shop
18128,0,28,4906,1,1,2013,19,1,1,0,1799.0,1799.0,1.0,6058.0,377.0,3840464.53,6058.0,8.0
47143,0,15,3686,1,1,2013,19,1,1,0,899.0,899.0,1.0,2353.0,157.0,1769615.00,2353.0,1.0
107443,0,37,22087,1,1,2013,83,1,1,0,79.0,158.0,2.0,1959.0,26.0,1312509.00,1959.0,11.0


In [472]:
# Get missing columns in the training test
missing_cols = set(train_df.columns) - set(test.columns )

In [473]:
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
test = test[train_df.columns]

In [474]:
test.head(2)

,ID,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_day,day,month,...,weekday,weekend,item_cnt_month,cat_cnt_month,cat_cnt_month,cat_cnt_month,cat_cnt_month,item_cnt_month_shop,revenue_month,global_cnt_month_shop
0,0,0,0,5,5037,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,5,5320,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [475]:
test[dep] = 0
joined_test = test[cat_vars+contin_vars+[dep]].copy()

In [476]:
joined_test.head(3)

,date_block_num,shop_id,item_id,day,month,year,item_category_id,holiday,weekday,weekend,...,revenue_day,item_cnt_day,item_cnt_month,cat_cnt_month,cat_cnt_month,cat_cnt_month,cat_cnt_month,revenue_month,global_cnt_month_shop,item_cnt_month_shop
0,0,5,5037,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,5,5320,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5,5233,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [477]:
joined.head(3)

,date_block_num,shop_id,item_id,day,month,year,item_category_id,holiday,weekday,weekend,item_price,revenue_day,item_cnt_day,item_cnt_month,cat_cnt_month,revenue_month,global_cnt_month_shop,item_cnt_month_shop
18128,0,28,4906,1,1,2013,19,1,1,0,1799.0,1799.0,1.0,6058.0,377.0,3840464.53,6058.0,8.0
47143,0,15,3686,1,1,2013,19,1,1,0,899.0,899.0,1.0,2353.0,157.0,1769615.00,2353.0,1.0
107443,0,37,22087,1,1,2013,83,1,1,0,79.0,158.0,2.0,1959.0,26.0,1312509.00,1959.0,11.0


In [478]:
#for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [479]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

In [493]:
df, y, nas, mapper = proc_df(joined, 'item_cnt_month_shop', do_scale=True)
#yl = np.log(y)
y1 = [np.log(i) if i!= 0 else 0 for i in y]
y1 = np.array(y1)

/Users/youcefdjeddar/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [428]:
df.head(2)

,date_block_num,day,month,holiday,weekday,weekend,item_price,revenue_day,item_cnt_day,item_cnt_month,cat_cnt_month,revenue_month,global_cnt_month_shop,shop_id,item_id,item_category_id,year
18128,-2.12439,-1.679279,-1.638782,5.923266,-1.095239,-0.710088,0.420456,0.054771,-0.096962,1.075885,0.316691,0.347773,1.075885,-0.252111,-0.747455,-1.158649,-1.48843
47143,-2.12439,-1.679279,-1.638782,5.923266,-1.095239,-0.710088,-0.072052,-0.069470,-0.096962,-0.304140,-0.272115,-0.511752,-0.304140,-1.041635,-0.941109,-1.158649,-1.48843


In [401]:
#joined_test = joined_test.set_index("shop_id")

In [482]:
#df_test, _, nas, mapper = proc_df(joined_test, 'item_cnt_month_shop', do_scale=True, mapper=mapper, na_dict=nas)

In [483]:
df.head(2)

,date_block_num,shop_id,item_id,day,month,year,item_category_id,holiday,weekday,weekend,item_price,revenue_day,item_cnt_day,item_cnt_month,cat_cnt_month,revenue_month,global_cnt_month_shop
18128,-2.12439,-0.252111,-0.747455,-1.679279,-1.638782,-1.488207,-1.158649,5.923266,-1.095239,-0.710088,0.420456,0.054771,-0.096962,1.075885,0.316691,0.347773,1.075885
47143,-2.12439,-1.041635,-0.941110,-1.679279,-1.638782,-1.488207,-1.158649,5.923266,-1.095239,-0.710088,-0.072052,-0.069470,-0.096962,-0.304140,-0.272115,-0.511752,-0.304140


In [484]:
df.columns

Index(['date_block_num', 'shop_id', 'item_id', 'day', 'month', 'year',
       'item_category_id', 'holiday', 'weekday', 'weekend', 'item_price',
       'revenue_day', 'item_cnt_day', 'item_cnt_month', 'cat_cnt_month',
       'revenue_month', 'global_cnt_month_shop'],
      dtype='object')

In [485]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [486]:
y1 = np.array(y1) ; y1

array([2.07944154, 0.        , 2.39789527, ..., 0.        , 0.        ,
       0.69314718])

In [438]:
#((val,trn), (y_val,y_trn)) = split_by_idx(val_idx, df.values, y1)

In [489]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [494]:
where_are_NaNs = isnan(y1)
y1[where_are_NaNs] = 0

In [495]:
n_valid = 214200  # same as Kaggle's test set size
n_trn = len(df)-n_valid
y_valid = 214200
n_trn = len(y1)-y_valid

In [496]:
X_train, y_valid = split_vals(df, n_trn)
X_valid, y_valid = split_vals(y1, n_trn)

In [498]:
assert len(X_train) == len(y_train)

In [455]:
m = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [456]:
m.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.99, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [466]:
preds = m.predict(X_valid)
m.score(X_train, y_train), m.score(X_valid, y_valid)

ValueError: Expected 2D array, got 1D array instead:
array=[2.08 0.   2.4  ... 1.1  0.69 0.69].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [499]:
X_train.columns

Index(['date_block_num', 'shop_id', 'item_id', 'day', 'month', 'year',
       'item_category_id', 'holiday', 'weekday', 'weekend', 'item_price',
       'revenue_day', 'item_cnt_day', 'item_cnt_month', 'cat_cnt_month',
       'revenue_month', 'global_cnt_month_shop'],
      dtype='object')

In [345]:
test.columns

Index(['ID', 'date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'revenue_day', 'day', 'month', 'year', 'item_name',
       'item_category_id', 'shop_name', 'holiday', 'weekday', 'weekend',
       'item_cnt_month', 'cat_cnt_month', 'item_cnt_month_shop',
       'revenue_month', 'global_cnt_month_shop'],
      dtype='object')

In [500]:
test = test[['date_block_num', 'shop_id', 'item_id', 'day', 'month', 'year', 'item_category_id', 'holiday',
            'weekday', 'weekend', 'item_price', 'revenue_day', 'item_cnt_day', 'item_cnt_month', 'cat_cnt_month'
             , 'revenue_month', 'global_cnt_month_shop']]

In [502]:
preds_test = m.predict(test)

ValueError: Number of features of the model must match the input. Model n_features is 17 and input n_features is 20 

In [348]:
preds_test

array([0.36647083, 0.36647083, 0.36647083, ..., 0.34264583, 0.34264583,
       0.34264583])

In [350]:
df_submission = pd.DataFrame(preds_test)
df_submission.columns = ['item_cnt_month']
#df_submission['ID'] = [i for i in range(len(test))]
cols = df_submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_submission = df_submission[cols]
df_submission.columns
df_submission.index.name = 'ID'

In [352]:
df_submission.head(2)

,item_cnt_month
ID,
0,0.366471
1,0.366471


In [353]:
df_submission.to_csv('/users/youcefdjeddar/downloads/submission_4.csv')

### Chronologically

In [336]:
X = train_df[['date_block_num', 'shop_id', 'item_id', 'day', 'month', 'year', 
            'item_category_id', 'holiday', 'weekday', 'weekend', 'item_price', 'revenue_day', 'item_cnt_day', 'item_cnt_month', 'cat_cnt_month',
     'revenue_month', 'global_cnt_month_shop']]
y = train_df['item_cnt_month_shop']

In [337]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 120000  # same as Kaggle's test set size
n_trn = len(X)-n_valid
raw_train, raw_valid = split_vals(train_df, n_trn)
X_train, X_valid = split_vals(X, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((1104439, 17), (1104439,), (120000, 17))

In [187]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
X_valid_scaled = min_max_scaler.fit_transform(X_valid)

/Users/youcefdjeddar/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/youcefdjeddar/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [188]:
X_train.head(3)

,date_block_num,shop_id,item_id,day,month,year,item_category_id,holiday,weekday,weekend,item_price,revenue_day,item_cnt_day,item_cnt_month,cat_cnt_month,revenue_month,global_cnt_month_shop
18128,0,28,4906,1,1,2013,19,1,1,0,1799.0,1799.0,1.0,6058.0,377.0,3840464.53,6058.0
47143,0,15,3686,1,1,2013,19,1,1,0,899.0,899.0,1.0,2353.0,157.0,1769615.00,2353.0
107443,0,37,22087,1,1,2013,83,1,1,0,79.0,158.0,2.0,1959.0,26.0,1312509.00,1959.0


In [189]:
X_train_scaled[0]

array([0.        , 0.45614035, 0.22026472, 0.        , 0.        ,
       0.        , 0.20987654, 1.        , 0.16666667, 0.        ,
       0.03038032, 0.03726724, 0.01673228, 0.37033841, 0.10458057,
       0.26038768, 0.37033841])

In [210]:
rfr = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 10, max_depth = 10, n_jobs = -1, oob_score = True).fit(X_train_scaled, y_train)
print("Training set score: {:.2f}".format(rfr.score(X_train_scaled, y_train)))
print("Validation set score: {:.2f}".format(rfr.score(X_valid_scaled, y_valid)))
#print("Test set score: {:.2f}".format(rfr.score(X_test_min_max, y_test)))
#%time rfr.fit(X_train_scaled, y_train)

Training set score: 0.94
Validation set score: 0.33


In [211]:
X_test_real.head(5)

,date_block_num,shop_id,item_id,day,month,year,item_category_id,holiday,weekday,weekend,item_price,revenue_day,item_cnt_day,item_cnt_month,cat_cnt_month,revenue_month,global_cnt_month_shop
0,0,5,5037,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,5,5320,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,5,5233,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,5,5232,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,5,5268,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [212]:
min_max_scaler = preprocessing.MinMaxScaler()
X_test_scaled = min_max_scaler.fit_transform(X_test_real)

/Users/youcefdjeddar/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [214]:
y_pred_real = rfr.predict(X_test_scaled)

In [215]:
df_submission = pd.DataFrame(y_pred_real)
df_submission.columns = ['item_cnt_month']
#df_submission['ID'] = [i for i in range(len(test))]
cols = df_submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_submission = df_submission[cols]
df_submission.columns

Index(['item_cnt_month'], dtype='object')

In [216]:
df_submission.index.name = 'ID'

In [217]:
df_submission.head(3)

,item_cnt_month
ID,
0,3.272228
1,3.272228
2,3.272228


In [219]:
df_submission.to_csv('/users/youcefdjeddar/downloads/submission_3.2.csv')

In [ ]:
plt.figure(figsize(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train, cmap=plt.cm.binary)
plt.show()   

In [ ]:
procs=[Categorify, Normalize]

In [ ]:
cat_vars = ['shop_id', 'item_id', 'day', 'month', 'year', 'item_category_id', 'holiday', 'weekday', 'weekend_indi']
count_vars = ['item_price', 'revenue']
dep_var = ['item_cnt_day']

In [ ]:
valid_idx = range(350000) ; valid_idx

In [ ]:
data = (TabularList.from_df(sales_merged, path=path, cat_names=cat_vars, cont_names=count_vars, procs=procs)
                .split_by_idx(valid_idx)
                .label_from_df(cols='item_cnt_day', label_cls=FloatList, log=True)
                .databunch())

In [ ]:
data

In [ ]:
data.train_ds

In [ ]:
learn = tabular_learner(data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, metrics=mean_squared_logarithmic_error)

In [ ]:
learn.lr_find()
learn.recorder.plot()